In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_selection import chi2, RFE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


In [15]:
df = pd.read_csv('raw_data.csv')
print("Dữ liệu ban đầu:")
print(df.head())

Dữ liệu ban đầu:
       ID      Nganh Gioi tinh  Doi tuong Khu vuc Khoi TS  Diem TS  Hoc Ky  \
0  A00001  Ngân hàng       Nam          0       3     A00     26.0       5   
1  A00001  Ngân hàng       Nam          0       3     A00     26.0       6   
2  A00001  Ngân hàng       Nam          0       3     A00     26.0       7   
3  A00001  Ngân hàng       Nam          0       3     A00     26.0       8   
4  A00002  Ngân hàng       Nam          1       1     A00     30.0       5   

   DKHK  TBHK   TCTL  TBTL XLHV  Diem TN                  KET QUA  
0    19  3.50   89.0  2.89  NaN     2.98   Tốt nghiệp Khá - 4 năm  
1    18  2.67  107.0  2.85  NaN     2.98   Tốt nghiệp Khá - 4 năm  
2    18  3.67  125.0  2.98  NaN     2.98   Tốt nghiệp Khá - 4 năm  
3    16  3.00  141.0  2.98  NaN     2.98   Tốt nghiệp Khá - 4 năm  
4    19  3.17   89.0  3.11  NaN     3.23  Tốt nghiệp Giỏi - 4 năm  


In [16]:
print("\nThông tin dữ liệu:")
print(df.info())
print("\nSố lượng giá trị thiếu theo cột:")
print(df.isnull().sum())


Thông tin dữ liệu:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198374 entries, 0 to 198373
Data columns (total 15 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   ID         198374 non-null  object 
 1   Nganh      198374 non-null  object 
 2   Gioi tinh  198374 non-null  object 
 3   Doi tuong  198374 non-null  int64  
 4   Khu vuc    198374 non-null  object 
 5   Khoi TS    198374 non-null  object 
 6   Diem TS    198229 non-null  float64
 7   Hoc Ky     198374 non-null  int64  
 8   DKHK       198374 non-null  int64  
 9   TBHK       198374 non-null  float64
 10  TCTL       198183 non-null  float64
 11  TBTL       198374 non-null  float64
 12  XLHV       15453 non-null   object 
 13  Diem TN    198255 non-null  float64
 14  KET QUA    198255 non-null  object 
dtypes: float64(5), int64(3), object(7)
memory usage: 22.7+ MB
None

Số lượng giá trị thiếu theo cột:
ID                0
Nganh             0
Gioi tinh         0
Doi tuong 

In [17]:
import re

def label_target(x):
    if not isinstance(x, str) or "Tốt nghiệp" not in x:
        return 0
    match = re.search(r'(\d+,\d+|\d+)\s*năm', x)
    if match:
        years = float(match.group(1).replace(',', '.'))
        return 1 if years == 4.0 else 0
    else:
        return 0

df['target'] = df['KET QUA'].apply(label_target)


In [18]:
categorical_cols = ['Nganh', 'Gioi tinh', 'Doi tuong', 'Khu vuc', 'Khoi TS']
df[categorical_cols] = df[categorical_cols].astype(str)

df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

num_cols = ['Diem TS', 'Hoc Ky', 'DKHK', 'TBHK', 'TCTL', 'TBTL', 'Diem TN']
scaler = StandardScaler()
df_encoded[num_cols] = scaler.fit_transform(df_encoded[num_cols])

print("\nDữ liệu sau khi tiền xử lý:")
print(df_encoded.head())


Dữ liệu sau khi tiền xử lý:
       ID   Diem TS    Hoc Ky      DKHK      TBHK      TCTL      TBTL XLHV  \
0  A00001  1.786793  0.086646  0.251934  1.024862  0.275946  0.449943  NaN   
1  A00001  1.786793  0.479386  0.095738  0.158309  0.703191  0.379144  NaN   
2  A00001  1.786793  0.872126  0.095738  1.202349  1.130436  0.609241  NaN   
3  A00001  1.786793  1.264867 -0.216655  0.502842  1.510209  0.609241  NaN   
4  A00002  3.441126  0.086646  0.251934  0.680329  0.275946  0.839338  NaN   

    Diem TN                  KET QUA  ...  Doi tuong_6  Doi tuong_7  \
0  0.469105   Tốt nghiệp Khá - 4 năm  ...        False        False   
1  0.469105   Tốt nghiệp Khá - 4 năm  ...        False        False   
2  0.469105   Tốt nghiệp Khá - 4 năm  ...        False        False   
3  0.469105   Tốt nghiệp Khá - 4 năm  ...        False        False   
4  0.725510  Tốt nghiệp Giỏi - 4 năm  ...        False        False   

   Khu vuc_2  Khu vuc_2NT  Khu vuc_3  Khoi TS_A01  Khoi TS_C00  Khoi TS_D01

In [19]:
X = df_encoded.drop(['ID', 'KET QUA', 'target', 'XLHV'], axis=1)
y = df_encoded['target']

In [21]:
from sklearn.impute import SimpleImputer

print(X.isnull().sum())

imputer = SimpleImputer(strategy='mean')
X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

model_lr = LogisticRegression(max_iter=1000)
rfe = RFE(model_lr, n_features_to_select=10)
rfe.fit(X_imputed, y)
selected_features = X.columns[rfe.support_]
print("\nĐặc trưng được chọn bởi RFE:")
print(selected_features)


Diem TS                         145
Hoc Ky                            0
DKHK                              0
TBHK                              0
TCTL                            191
TBTL                              0
Diem TN                         119
Nganh_Kinh doanh chứng khoán      0
Nganh_Kinh doanh quốc tế          0
Nganh_Kế toán doanh nghiệp        0
Nganh_Luật kinh tế                0
Nganh_Ngân hàng                   0
Nganh_Ngôn ngữ Anh                0
Nganh_Quản trị doanh nghiệp       0
Nganh_Quản trị kinh doanh         0
Nganh_Quản trị marketing          0
Nganh_Thanh toán quốc tế          0
Nganh_Tài chính                   0
Gioi tinh_Nữ                      0
Doi tuong_1                       0
Doi tuong_3                       0
Doi tuong_4                       0
Doi tuong_5                       0
Doi tuong_6                       0
Doi tuong_7                       0
Khu vuc_2                         0
Khu vuc_2NT                       0
Khu vuc_3                   